<h1 style="background-color:#00008B; font-family:'Brush Script MT',cursive;color:#DC143C;font-size:200%; text-align:center;border-radius: 50% 20% / 10% 40%">Long-tailed Learning</h1>

"Long-tailed learning, one of the most challenging problems in visual recognition, aims to train well-performing models from a large number of images that follow a long-tailed class distribution."

https://paperswithcode.com/task/long-tail-learning

#当前位置：网站首页>Yan Shuicheng and Feng Jiashi's team released the first review of deep long tail learning!

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSlB8opuohbo0z2YKKvhy1Ir_lFYVFkrf4ugQ&usqp=CAU)chowdera.com

https://chowdera.com/2021/11/20211128120204616d.html

https://arxiv.org/pdf/2110.04596.pdf

I will try to develop that subject on another Kaggle Notebook, since it's so compelling.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

import imageio
import matplotlib as mpl
import matplotlib.image as mpimg
from PIL import Image, ImageOps
import scipy.ndimage as ndi

In [ ]:
#Code by Ventakumar R https://www.kaggle.com/venkatkumar001/hfp-2-eda-tensorflow/notebook

import json, codecs

with codecs.open("../input/herbarium-2022-fgvc9/train_metadata.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    train_meta = json.load(f)
    
with codecs.open("../input/herbarium-2022-fgvc9/test_metadata.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    test_meta = json.load(f)

In [ ]:
#Code by Ventakumar R https://www.kaggle.com/venkatkumar001/hfp-2-eda-tensorflow/notebook

display(train_meta.keys())

In [ ]:
#Code by Ventakumar R https://www.kaggle.com/venkatkumar001/hfp-2-eda-tensorflow/notebook

Ltailed = pd.DataFrame(train_meta['annotations'])
#train_cat.columns = [ 'category_id', 'scientificName','family', 'genus']
display(Ltailed)

After having another idea. Let's go to Plan B. 

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

#here we need to get to the data
dirname = '/kaggle/input/herbarium-2022-fgvc9/train_images'
dir_004 = os.path.join(dirname, '004')
dir_004_13 = os.path.join(dir_004, '13')

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

os.listdir(dir_004)

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

os.listdir(dir_004_13)

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

def plot_imgs_one_dir(item_dir, num_imgs=25):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs][:num_imgs]

    plt.figure(figsize=(10, 10))
    for idx, img_path in enumerate(item_files):
        plt.subplot(5, 5, idx+1)

        img = plt.imread(img_path)
        plt.imshow(img)

    plt.tight_layout()

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

plot_imgs_one_dir(dir_004_13)

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

def plot_img_hist_3channel(item_dir, num_img=6):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs][:num_img]

    #plt.figure(figsize=(10, 10))
    for idx, img_path in enumerate(item_files):
        fig1 = plt.figure(idx,figsize=(10, 10))
        fig1.add_subplot(2, 2, 1)
        img = mpimg.imread(img_path, )
        plt.imshow(img)
        fig1.add_subplot(2, 2, 2)
        plt.hist(img.ravel(), bins = 256, color = 'orange')
        plt.hist(img[:, :, 0].ravel(), bins = 256, color = 'red')
        plt.hist(img[:, :, 1].ravel(), bins = 256, color = 'green')
        plt.hist(img[:, :, 2].ravel(), bins = 256, color = 'blue')
        plt.xlabel('Intensity')
        plt.ylabel('Count')
        plt.legend(['Total', 'Red Channel', 'Green Channel', 'Blue Channel'])
        plt.show()
    
    plt.tight_layout()

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

plot_img_hist_3channel(dir_004_13)

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

os.mkdir('/kaggle/working/work_imgs')

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

dir_output = os.path.join('/kaggle/working/','work_imgs');

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

def  hist_equal(path_from, path_to, stop=4):
    i=1
    files = os.listdir(path_from)
    
    for file in files[:stop]: 
        try:
            file_dir = os.path.join(path_from, file)
            file_dir_save = os.path.join(path_to, file)
            img = Image.open(file_dir)
            img = ImageOps.equalize(img)
            #img = img.convert("RGB") #konwersja z RGBA do RGB, usuniecie kanału alfa zeby zapisać do jpg
            img.save(file_dir_save) 
            i=i+1
        except:
            continue

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

hist_equal(dir_004_13, dir_output)

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

plot_img_hist_3channel(dir_output, 4)

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

def plot_cdf_comparison(item_dir_before,item_dir_after, num_img=1):
    all_item_dirs = os.listdir(item_dir_before)
    item_files_before = [os.path.join(item_dir_before, file) for file in all_item_dirs][:num_img]
    item_files_after = [os.path.join(item_dir_after, file) for file in all_item_dirs][:num_img]
  
  #plt.figure(figsize=(10, 10))
    for idx, img_path in enumerate(item_files_before):
        im_b = imageio.imread(img_path)
        hist_b = ndi.histogram(im_b, min=0, max=255, bins=256)
        cdf_b = hist_b.cumsum() / hist_b.sum()
        
        img_path_a = item_files_after[idx]
        im_a = imageio.imread(img_path_a)
        hist_a = ndi.histogram(im_a, min=0, max=255, bins=256)
        cdf_a = hist_a.cumsum() / hist_a.sum()

        fig1 = plt.figure(idx,figsize=(10, 10))
        fig1.add_subplot(2, 4, 1)
        img_b = mpimg.imread(img_path, )
        plt.title("Before. {}".format(idx))
        plt.imshow(img_b, cmap='gray')
        fig1.add_subplot(2, 4, 4)
        plt.title("CDF comparison")
        plt.plot(cdf_b)
        
        fig2 = plt.figure(idx,figsize=(10, 10))
        fig2.add_subplot(2, 4, 2)
        img_a = mpimg.imread(img_path_a, )
        plt.title("Before. {}".format(idx))
        plt.imshow(img_a)
        fig1.add_subplot(2, 4, 4)
        plt.plot(cdf_a)
        plt.legend(['CDF before', 'CDF after'])

    plt.tight_layout()

In [ ]:
#Code by Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

plot_cdf_comparison(dir_004_13, dir_output, 4)

#Acknowledgements:

Wojciech Łabuñski https://www.kaggle.com/wojciech1103/herbarium-2020-a-little-bit-fun-with-images/notebook

Ventakumar R https://www.kaggle.com/venkatkumar001/hfp-2-eda-tensorflow/notebook

![](http://bair.berkeley.edu/static/blog/oltr/teaser.png)bair.berkeley.edu